In [119]:
#imports
import duckdb
import warnings
import pandas as pd

In [120]:
#Settings
pd.set_option('display.max_columns', None)
warnings.simplefilter(action='ignore', category=FutureWarning) 

In [121]:
# create a connection to a file called 'database.db'
con = duckdb.connect('database.db')

In [122]:
con.sql("show tables")

┌────────────┐
│    name    │
│  varchar   │
├────────────┤
│ categories │
│ podcasts   │
│ reviews    │
└────────────┘

In [123]:
#describe podcasts table
con.sql("describe podcasts")

┌────────────────┬─────────────┬─────────┬─────────┬─────────┬───────┐
│  column_name   │ column_type │  null   │   key   │ default │ extra │
│    varchar     │   varchar   │ varchar │ varchar │ varchar │ int32 │
├────────────────┼─────────────┼─────────┼─────────┼─────────┼───────┤
│ podcast_id     │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ itunes_id      │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ slug           │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ itunes_url     │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ title          │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ author         │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ description    │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ average_rating │ DOUBLE      │ YES     │ NULL    │ NULL    │  NULL │
│ ratings_count  │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ scraped_at     │ TIMESTAMP   │ YES     │ NULL    │ NULL    │  NULL │
├─────

In [124]:
#describe categories table
con.sql("describe categories")

┌─────────────┬─────────────┬─────────┬─────────┬─────────┬───────┐
│ column_name │ column_type │  null   │   key   │ default │ extra │
│   varchar   │   varchar   │ varchar │ varchar │ varchar │ int32 │
├─────────────┼─────────────┼─────────┼─────────┼─────────┼───────┤
│ podcast_id  │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ itunes_id   │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ category    │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
└─────────────┴─────────────┴─────────┴─────────┴─────────┴───────┘

In [125]:
#describe reviews table
con.sql("describe reviews")


┌─────────────┬──────────────────────────┬─────────┬─────────┬─────────┬───────┐
│ column_name │       column_type        │  null   │   key   │ default │ extra │
│   varchar   │         varchar          │ varchar │ varchar │ varchar │ int32 │
├─────────────┼──────────────────────────┼─────────┼─────────┼─────────┼───────┤
│ podcast_id  │ VARCHAR                  │ YES     │ NULL    │ NULL    │  NULL │
│ title       │ VARCHAR                  │ YES     │ NULL    │ NULL    │  NULL │
│ content     │ VARCHAR                  │ YES     │ NULL    │ NULL    │  NULL │
│ rating      │ INTEGER                  │ YES     │ NULL    │ NULL    │  NULL │
│ author_id   │ VARCHAR                  │ YES     │ NULL    │ NULL    │  NULL │
│ created_at  │ TIMESTAMP WITH TIME ZONE │ YES     │ NULL    │ NULL    │  NULL │
└─────────────┴──────────────────────────┴─────────┴─────────┴─────────┴───────┘

In [126]:
#Look into data for reviews
con.sql("select * from reviews limit 5")

┌──────────────────────┬──────────────────────┬──────────────────────┬────────┬─────────────────┬──────────────────────┐
│      podcast_id      │        title         │       content        │ rating │    author_id    │      created_at      │
│       varchar        │       varchar        │       varchar        │ int32  │     varchar     │ timestamp with tim…  │
├──────────────────────┼──────────────────────┼──────────────────────┼────────┼─────────────────┼──────────────────────┤
│ 52e3d2c4fab4e80a8b…  │ Argh                 │ What the heck is t…  │      1 │ a23655ce5565f32 │ 2019-04-30 21:24:0…  │
│ 52e3d2c4fab4e80a8b…  │ Miss gross overrat…  │ The Howard stern i…  │      1 │ e5d461c5f91c8d2 │ 2019-06-07 10:25:3…  │
│ 52e3d2c4fab4e80a8b…  │ I love Terri and F…  │ People get really …  │      5 │ 806cc75442c8f3d │ 2019-05-15 05:42:5…  │
│ 52e3d2c4fab4e80a8b…  │ Fresh Air subscrip…  │ I’ve been a subscr…  │      4 │ 3e1e7e8417a3e85 │ 2019-05-05 22:39:1…  │
│ 52e3d2c4fab4e80a8b…  │ Climate

In [127]:
#Total Number of reviews
con.sql("select count(*) as total_number_of_reviews from reviews")

┌─────────────────────────┐
│ total_number_of_reviews │
│          int64          │
├─────────────────────────┤
│                 5607021 │
└─────────────────────────┘

In [128]:
con.sql("select * from podcasts where average_rating >= 0")

┌──────────────────────┬────────────┬──────────────────────┬───┬────────────────┬───────────────┬──────────────────────┐
│      podcast_id      │ itunes_id  │         slug         │ … │ average_rating │ ratings_count │      scraped_at      │
│       varchar        │  varchar   │       varchar        │   │     double     │    varchar    │      timestamp       │
├──────────────────────┼────────────┼──────────────────────┼───┼────────────────┼───────────────┼──────────────────────┤
│ f201172864310534ae…  │ 1000016089 │ rahdo-talks-through  │ … │            4.6 │ 60            │ 2019-07-08 06:15:4…  │
│ 3cfef94cfccf27eb50…  │ 1000035707 │ originall            │ … │            3.7 │ 3             │ 2019-07-08 08:49:4…  │
│ 2f2205dcc248024b0a…  │ 1000068299 │ the-big-podcast-wi…  │ … │            4.6 │ 1.3K          │ 2019-07-08 23:01:4…  │
│ 9fc35a81647e391f93…  │ 1000081327 │ galaktacus           │ … │            5.0 │ 20            │ 2019-07-09 16:22:0…  │
│ ce2c3a68dda3560067…  │ 1000095

In [129]:
#Unique authors contributing reviews
con.sql("select count(DISTINCT author_id) as unique_authors from reviews")

┌────────────────┐
│ unique_authors │
│     int64      │
├────────────────┤
│        3256501 │
└────────────────┘

In [130]:
authors = con.sql("select author_id,count(author_id) as count from reviews group by author_id")

In [131]:
#Total Number of reviews
authors.df()["count"].sum()

5607021

In [132]:
authors.df()["count"].mean()


1.7217931147572194

In [133]:
#Top 5 authors
authors.df().sort_values('count',ascending=False).head(5)

,author_id,count
2106925,d3307adeffa285c,1681
866232,791f376aca3c08d,1303
2767141,8ea31f030185a88,1248
1346059,96a3f0358e7808d,1229
920166,04c3b6defe22fb1,1094


In [134]:
authors.df()["count"].mean()

1.7217931147572194

In [135]:
one_total = 0
non_one_total = 0
non_one_total_ratings = 0
for (colname,colval) in authors.df().iteritems():
    if colname == "count":
        for x in colval:
            if x == 1:
                one_total += 1
            else:
                non_one_total += 1
                non_one_total_ratings += x
print(f'Total Authors with only 1 rating: {one_total:,}')
print(f'Total Authors with more than 1 rating: {non_one_total:,}')
print(f'Average rating for Authors with more than 1 rating: {non_one_total_ratings/non_one_total:,}')
print(f'Total Ratings: {non_one_total+one_total:,}') 

Total Authors with only 1 rating: 2,400,918
Total Authors with more than 1 rating: 855,583
Average rating for Authors with more than 1 rating: 3.747272912154636
Total Ratings: 3,256,501


In [136]:
#Bottom 5 authors
authors.df().sort_values('count',ascending=True).head(5)

,author_id,count
1628250,bf76946f3ff902f,1
2004268,5703109137c25e9,1
2004269,48fe9fa82d7e3fd,1
2004270,f144e074e9f647e,1
2004271,0d879b725bc9de7,1


In [137]:
#Look into data for reviews that have an average rating
rated_podcasts = con.sql("select podcast_id,title,average_rating,ratings_count from podcasts where average_rating >= 0")
rated_podcasts

┌──────────────────────┬──────────────────────────────────────────────────────────────┬────────────────┬───────────────┐
│      podcast_id      │                            title                             │ average_rating │ ratings_count │
│       varchar        │                           varchar                            │     double     │    varchar    │
├──────────────────────┼──────────────────────────────────────────────────────────────┼────────────────┼───────────────┤
│ f201172864310534ae…  │ Rahdo Talks Through                                          │            4.6 │ 60            │
│ 3cfef94cfccf27eb50…  │ OriginALL                                                    │            3.7 │ 3             │
│ 2f2205dcc248024b0a…  │ The Big Podcast With Shaq                                    │            4.6 │ 1.3K          │
│ 9fc35a81647e391f93…  │ Galaktacus                                                   │            5.0 │ 20            │
│ ce2c3a68dda3560067…  │ No Extr

In [138]:
total_podcast_ratings = 0
rated_podcasts_df = rated_podcasts.df()
podcast_ratings = []
for (colname,colval) in rated_podcasts_df.iteritems():
    if colname == "ratings_count":
        for x in colval:
            if  "K" not in x:
                total_podcast_ratings += int(x)
                podcast_ratings.append(int(x))
            else:
                formatted = float(x[0:x.find('K')])
                total_podcast_ratings += (1000 * formatted)   
                podcast_ratings.append(int(1000 * formatted))
rated_podcasts_df.drop("ratings_count", axis = 1, inplace = True)
rated_podcasts_df["ratings_count"] = podcast_ratings

In [139]:
rated_podcasts_df.shape[0]

82988

In [140]:
#First five values in rated podcasts data frame
rated_podcasts_df.head()


,podcast_id,title,average_rating,ratings_count
0,f201172864310534ae6b6c4f8ae9b8cc,Rahdo Talks Through,4.6,60
1,3cfef94cfccf27eb504bd7a6c7459636,OriginALL,3.7,3
2,2f2205dcc248024b0a3be4665f38e33a,The Big Podcast With Shaq,4.6,1300
3,9fc35a81647e391f93211b5d11973c83,Galaktacus,5.0,20
4,ce2c3a68dda35600671acd32a1c15fc5,No Extra Words one person's search for story,5.0,11


In [141]:
#Most rated podcasts
rated_podcasts_df.sort_values('ratings_count',ascending=False).head(5)

,podcast_id,title,average_rating,ratings_count
6022,608c549bece62f009d74396cacbb54c3,Pardon My Take,4.9,56000
12424,71819bcee712a1ad4aaa97fb8f1a5f09,Pod Save America,4.7,51000
57663,57008fd80e7d710ac9283f3e64e9c9a7,Stuff You Should Know,4.6,39000
12588,de8caf3e4e8178b0280a52c7b3d51486,Small Town Murder,4.9,31000
14355,e50884ba76af05b03683b693e2b3e8f7,Wait Wait...Don't Tell Me!,4.7,21000


In [142]:
#Podcasts with top 5 average rating
rated_podcasts_df.sort_values('average_rating',ascending=False).head(5)

,podcast_id,title,average_rating,ratings_count
41494,134a2d138569aec6b59dcf0b8ee4fb72,Cerealcast,5.0,1
42157,c020d87fec92abe047e6b392e4e74e6b,Global Denmark Podcast,5.0,9
40733,e00550eb3fe66294b4e1f78ab9d2bc6d,Faking It: An SFX Podcast,5.0,4
73774,239d9a6b1de44631a6a23122a7f956d5,NSVille,5.0,3
40735,66c41ef09462ddc13a76b31a81f977bd,Under The First Floor,5.0,5


In [143]:
#Top rated podcasts with most ratings
rated_podcasts_df.sort_values(['average_rating', 'ratings_count'],
              ascending = [False, False]).head(5)

,podcast_id,title,average_rating,ratings_count
41479,2cb1f92d25e260d43d7c7131b78151e6,The Corp,5.0,9300
51386,e2bd23c58ea63ca8c8c6f6626fa438ca,The TryPod,5.0,8500
72481,dc70a11a1927080ebab98be779ca4322,Grumpy Old Geeks,5.0,6300
25308,a066be63e15cb4125eb83d4f72f855eb,The Anthropocene Reviewed,5.0,4100
51435,6bd787536fb3505dc3c6e091829bfa98,Cal's Week in Review,5.0,4000


In [144]:
con.sql("select * from categories").df().head(5)

,podcast_id,itunes_id,category
0,943d1c4349913bde09ef82b69f83cfdf,1446474395,education
1,f53f27d03cdc5040ab4fd2ea30e3dd4f,840646770,society-culture
2,49fa5f1097ae9046ce1a0c56e31b533c,1541959455,arts-performing-arts
3,dbe2dc5aa097523aa294a3b659293af4,1146609744,tv-film
4,2349ed9d26f16029cfc69902bf319ca5,1475252651,education


In [145]:
#Unique Categories
con.sql("select count(distinct category) as unique_categories from categories").df().head(5)

,unique_categories
0,110


In [146]:
categories = con.sql("select category,count(category) as count from categories group by category").df()
categories.head()

,category,count
0,business,211977
1,tv-film,68639
2,education-courses,40668
3,society-culture-personal-journals,75815
4,arts-books,90382


In [147]:
#Top 5 most common categories
categories.sort_values('count',ascending=False).head(5)

,category,count
27,education,311924
12,society-culture,301139
6,arts,227713
0,business,211977
8,religion-spirituality,206257


In [148]:
#Bottom 5 most common categories
categories.sort_values('count',ascending=True).head(5)

,category,count
108,sports-volleyball,265
107,sports-swimming,304
82,sports-tennis,563
105,sports-rugby,648
104,sports-cricket,755
